In [1]:
from lxml import html
import requests
import pandas as pd
import time
from datetime import datetime
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-09 09:07:12


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15

Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB  = Project_DIR + '\\2.Project_GDB.gdb'
ContractsDir = Project_DIR + '\\4.Contracts_AnalysisFiles'
DocumentsDir = Project_DIR + '\\5.Documents'
PythonDir    = Project_DIR + '\\6.Python_Codes'
ReceivedDir  = Project_DIR + '\\7.ReceivedData'
XML_Dir      = ReceivedDir + '\\HSIP Candidate Applications'


# IRIS Dataset
IRISPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\5.0_Roadway_Layer\IRIS'
IRISPath = r'C:\Users\mr068144\Downloads\IL Data\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in Years}
IRIS_Tab   = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
IRIS_Tab[2012] = IRIS_Tab[2013]

# Intersections Dataset
IntPG = r'C:\Users\mr068144\Downloads\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG'
Intersections = {year:IntPG for year in Years}

#Crash Dataset
CrashData  = {y:os.path.abspath(r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\IL_Crash.gdb\\' +
                          'CrashExtract_' + str(y) + '_GIS') for y in Years}

Fields = ['AADT',
          'O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH',
          'I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH',
          'LNS','LN_WTH','MED_TYP','MED_WTH','SURF_WTH','URBAN','PG','Radius','CurveLen']
#Fields = ['AADT','URBAN','PG']
RouteID = 'INVENTORY'
BMP = 'BEG_STA'
EMP = 'END_STA'

# Geocoded projects
HSIP_Int  = Project_GDB + '\\WPPS_Int'
HSIP_Seg  = Project_GDB + '\\WPPS_Seg'

HSIP_Path    = ReceivedDir + '\\HSIP_AllApprovedProjects.xlsm'
WPPS_Ob_Path = ReceivedDir + '\\HSIP Contract and Obligation Data.xlsx'
WPPS_Pr_Path = ReceivedDir + '\\HSIP Project Programmed Data.xlsx'
SPF_Data     = ReceivedDir  + '\\SPF Coefficients.xlsx'

2018-11-09 09:07:12


In [3]:
def GetAllLinks(url):
    cont = requests.get(url).content
    tree = html.fromstring(cont)
    title = tree.xpath('/html/body')
    LinkDF = pd.DataFrame(columns=['Link','Type'])
    i = 0
    for l in title[0]:
        for a in l:
            try:
                href = a.attrib['href']
                if a.text== '[To Parent Directory]':
                    continue
                if href[-1]=='/':
                    i += 1
                    LinkDF.loc[i]=['http://apps.dot.illinois.gov{}'.format(href),'Dir']
                if href.find('.')!=-1:
                    i += 1
                    LinkDF.loc[i]=['http://apps.dot.illinois.gov{}'.format(href),href.split('.')[1].lower()]
            except:
                href = ''
    return(LinkDF)
def GetFileName(s):
    s = s[42:]
    sp = s.split('.')
    if len(sp)>1:
        return('{}.{}'.format(sp[-2].split('/')[-1],sp[-1]))
    else:
        return('')
def GetBidDate(s):
    try:
        s = s[42:]
        s.split('/')[0]
        m = int(s[:2])
        d = int(s[2:4])
        y = int(s[4:6]) + 2000
        #if m>=1 and m<=12 and d>=1 and d<=31 and y>=2002 and y<=2020:
        d = datetime(y,m,d)
        return(d)
    except:
        return(pd.NaT)
def GetContNum(s):
    try:
        s = s[42:]
        contnum,cont_ext = s.split('/')[1].split('-')
        return(contnum)
    except:
        pass
def GetContExt(s):
    try:
        s = s[42:]
        contnum,cont_ext = s.split('/')[1].split('-')
        return(cont_ext)
    except:
        pass

In [4]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Doc_DF = pd.read_csv('Contracts_index.csv')
Doc_DF = Doc_DF.drop(Doc_DF.columns[0], axis=1)
Doc_DF['Filename'] = [GetFileName(s) for s in Doc_DF.Link]
Doc_DF['Link2'] = [l[42:] for l in Doc_DF.Link]
Doc_DF['BidDate'] = [GetBidDate(s) for s in Doc_DF.Link]
Doc_DF['ContNum'] = [GetContNum(s) for s in Doc_DF.Link]
Doc_DF['ContExt'] = [GetContExt(s) for s in Doc_DF.Link]
print(Doc_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-09 09:07:12
(620781, 7)
2018-11-09 09:07:20


In [ ]:
# Create the index file
DF1 = GetAllLinks('http://apps.dot.illinois.gov/eplan/DESENV/')
DF1.to_csv('Contracts_index.csv')
while True in [t=='Dir' for t in DF1.Type]:
    for link in list(DF1[DF1.Type=='Dir']['Link']):
        try:
            DF2 = GetAllLinks(link)
            DF1 = DF1[DF1.Link!=link]
            DF1 = pd.concat([DF1,DF2])
            #print(link,DF1[DF1.Type=='Dir'].shape[0],DF1[DF1.Type!='Dir'].shape[0])
            DF1.to_csv('Contracts_index.csv')
        except:
            print(link,'Failed')
            pass

In [ ]:
# Create the status file
print(strftime("%Y-%m-%d %H:%M:%S"))
df = pd.read_html(ReceivedDir + '\\ContractStatus.html')
SDF = df[1]
SDF.columns = list(df[0].loc[0])
SDF.index = pd.MultiIndex.from_tuples([(c,a) for c,a in zip(SDF.Contract,SDF.Authorization)])
SDF = SDF[['Status','Status Date']]
SDF.columns = [c.replace(' ','_') for c in SDF.columns]
SDF.Status_Date = [datetime.strptime(d,'%m/%d/%Y') for d in SDF.Status_Date]
SDF = SDF[~SDF.index.duplicated(keep='first')]
print(SDF.shape)
baseurl = 'http://apps.dot.illinois.gov/ChangeOrder/ChangeOrder.aspx?contractNumber={}&authorizationNumber={}'.format
f = FloatProgress(min=0, max=len(SDF))
display(f)
for j,r in SDF.iterrows():
    try:
        cont = requests.get(baseurl(j[0],j[1])).content
        tree = html.fromstring(cont)
        title = tree.xpath('/html/body')
        df = pd.DataFrame()
        for i in range(0,15):
            df[title[0][3][2][0][2*i][0].text_content()[:-1].replace(' ','_')] = [title[0][3][2][0][2*i+1][0].text_content()]
        df.index = pd.MultiIndex.from_tuples([(df.loc[0,'Contract_Number'],df.loc[0,'Authorization_Number'])])
        SDF = pd.concat([SDF,df])
    except:
        print('Failed: {}, {}'.format(j[0],j[1]))
    f.value += 1
    SDF.to_csv('Contract_Descriptions.csv')
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Contract Authorization Numbers
print(strftime("%Y-%m-%d %H:%M:%S"))
df = pd.read_html(ReceivedDir + '\\ContractAuthNumbers\\ConstructionChangeOrders.html')
SDF1 = df[1]
SDF1.columns = list(df[0].loc[0])
SDF1.index = SDF1.Contract
#SDF = SDF[['Status','Status Date']]
SDF1.columns = [c.replace(' ','_') for c in SDF1.columns]
SDF1.Status_Date = [datetime.strptime(d,'%m/%d/%Y') for d in SDF1.Status_Date]
print(SDF1.shape)
df = pd.read_html(ReceivedDir + '\\ContractAuthNumbers\\HighwayAeronautics.html')
SDF2 = df[1]
SDF2.columns = list(df[0].loc[0])
SDF2.columns = [c.replace(' ','_') for c in SDF2.columns]
SDF2.index = SDF2.Contract
print(SDF2.shape)
df = pd.read_html(ReceivedDir + '\\ContractAuthNumbers\\EngineeringConsultant.html')
SDF3 = df[1]
SDF3.columns = list(df[0].loc[0])
SDF3.columns = [c.replace(' ','_') for c in SDF3.columns]
print(SDF3.shape)
df = pd.read_html(ReceivedDir + '\\ContractAuthNumbers\\EngineeringConsultantSupplement.html')
SDF4 = df[1]
SDF4.columns = list(df[0].loc[0])
SDF4.columns = [c.replace(' ','_') for c in SDF4.columns]
print(SDF4.shape)
df = pd.read_html(ReceivedDir + '\\ContractAuthNumbers\\LandAq.html')
SDF5 = df[1]
SDF5.columns = list(df[0].loc[0])
SDF5.columns = [c.replace(' ','_') for c in SDF5.columns]
print(SDF5.shape)
df = pd.read_html(ReceivedDir + '\\ContractAuthNumbers\\landAqAmendments.html')
SDF6 = df[1]
SDF6.columns = list(df[0].loc[0])
SDF6.columns = [c.replace(' ','_') for c in SDF6.columns]
print(SDF6.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

In [5]:
# Letting items
print(strftime("%Y-%m-%d %H:%M:%S"))
XLSList = [os.path.join(ReceivedDir+'\\PayItemReports', f) for f in os.listdir(ReceivedDir+'\\PayItemReports') 
           if os.path.isfile(os.path.join(ReceivedDir+'\\PayItemReports', f)) and 
           f.split('.')[1]=='xlsx']
Let_DF = pd.DataFrame()
for xls in XLSList:
    df = pd.read_excel(xls,header=[0,1])
    df.index.name = 'PAYITEM'
    df.columns = ['DESCRIPTION','UNIT','QUANTITY','COUNTY','DIST','CONTNUM','ITEMNUM','AWARDPRICE']
    s=os.path.basename(xls).split('.')[0][4:10]
    d= datetime(2000+int(s[4:6]),int(s[0:2]),int(s[2:4]),0,0,0)
    df['LetDate'] = d
    Let_DF = pd.concat([Let_DF,df])
Let_DF.QUANTITY = Let_DF.QUANTITY.astype(float)
Let_DF.AWARDPRICE = [hsmpy3.common.ConvertType(Value=a,Type='DOUBLE') for a in Let_DF.AWARDPRICE]
Let_DF['PayItem'] = list(Let_DF.index)
Let_DF['Amount'] = [q*p for q,p in zip(Let_DF.QUANTITY,Let_DF.AWARDPRICE)]
Let_DF.Amount = Let_DF.Amount.astype(float)
Let_DF = Let_DF.sort_values(['LetDate','PayItem','CONTNUM'])
print(Let_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-09 09:07:31
(445190, 11)
2018-11-09 09:09:11


In [126]:
# Clean Contract Descriptions
print(strftime("%Y-%m-%d %H:%M:%S"))
Cont_DF = pd.read_csv(PythonDir+'\\Contract_Descriptions.csv',index_col=[0])
Cont_DF.index.name = 'ContNum'
Cont_DF = Cont_DF[~pd.isnull(Cont_DF.Contract_Number)].sort_index()
Cont_DF.District = [hsmpy3.common.ConvertType(d,'SHORT') for d in Cont_DF.District]
Cont_DF.Authorization_Amount = [hsmpy3.il.ConvertHSIP(a,'currency') for a in Cont_DF.Authorization_Amount]
Cont_DF.Awarded_Contract_Value = [hsmpy3.il.ConvertHSIP(a,'currency') for a in Cont_DF.Awarded_Contract_Value]
Cont_DF.Status_Date = [hsmpy3.il.ConvertHSIP(a,'date') for a in Cont_DF.Status_Date]
Cont_DF = Cont_DF[['Authorization_Amount', 'Authorization_Number', 'Awarded_Contract_Value', 'Category_Code', 'Contract_Number', 'Contractor', 'County', 'Description', 'Determination_of_Germaneness', 'District', 'Project', 'Route', 'Section', 'Status', 'Status_Date']]
Cont_DF = Cont_DF.sort_index()
Cont_DF.shape
Cont_DF.to_csv('Contract_Descriptions_Cleaned.csv')
print(strftime("%Y-%m-%d %H:%M:%S"))

(7816, 15)

In [6]:
# Contract Decriptions
print(strftime("%Y-%m-%d %H:%M:%S"))
Desc_DF  = pd.read_csv('Contract_Descriptions_Cleaned.csv')
print(Desc_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-09 09:09:11
(7816, 16)
2018-11-09 09:09:11


In [7]:
# Contract Amounts
print(strftime("%Y-%m-%d %H:%M:%S"))
XLSList = [os.path.join(ReceivedDir+'\\Contracts', f) for f in os.listdir(ReceivedDir+'\\Contracts') 
           if os.path.isfile(os.path.join(ReceivedDir+'\\Contracts', f)) and 
           f.split('.')[1]=='xls']
Pay_DF = pd.DataFrame()
for xls in XLSList:
    df = pd.read_excel(xls)
    df['ContNum'] = os.path.basename(xls).split('.')[0]
    Pay_DF = pd.concat([Pay_DF,df])
print(Pay_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-09 09:09:11
(39284, 10)
2018-11-09 09:09:29


In [122]:
# Updating contract PayItems based on Letting Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Let_DF.PayItem = Let_DF.PayItem.astype(str)
Let_DF.DESCRIPTION = Let_DF.DESCRIPTION.astype(str)
Let_DF = Let_DF.sort_values(by=['PayItem','LetDate'],ascending=[True,False])
Let_DF1 = Let_DF.drop_duplicates(['PayItem','DESCRIPTION'])
Let_DF1 = Let_DF1[['DESCRIPTION','UNIT','PayItem']]
Let_DF1.index = Let_DF1.PayItem.astype(str)
Let_DF1.PayItem = Let_DF1.PayItem.astype(str)
Let_DF1 = Let_DF1.sort_index()
Let_DF1['DescLen'] = [len(s) for s in Let_DF1.DESCRIPTION]
Let_DF1 = Let_DF1.sort_values(['PayItem','DescLen'],ascending=[True,False])
Let_DF1 = Let_DF1.drop_duplicates('PayItem')

Pay_DF.index = pd.MultiIndex.from_tuples([(str(c),str(p)) for c,p in zip(Pay_DF.ContNum,Pay_DF.PAY_ITEM)],names=['ContNum','PayItem'])
Pay_DF = Pay_DF[['PAY_ITEM_CODE', 'UNIT_PRICE', 'AWARDED_QTY', 'AWARDED_AMT', 'ADJUSTED_QTY', 'ADJUSTED_AMT', 'COMPLETED_QTY', 'COMPLETED_AMT', 'ContNum','PAY_ITEM']]
Pay_DF['Desc2'] = list(Let_DF1.loc[list(Pay_DF.index.droplevel(0)),'DESCRIPTION'])
Pay_DF['Unit'] = list(Let_DF1.loc[list(Pay_DF.index.droplevel(0)),'UNIT'])
Pay_DF.loc[pd.isnull(Pay_DF.PAY_ITEM_CODE),'PAY_ITEM_CODE'] = Pay_DF.loc[pd.isnull(Pay_DF.PAY_ITEM_CODE),'Desc2']
Pay_DF = Pay_DF.drop('Desc2',axis=1)
Pay_DF = Pay_DF.sort_values(['ContNum','COMPLETED_AMT'],ascending=[True,False])
Pay_DF = Pay_DF[['PAY_ITEM_CODE', 'Unit', 'UNIT_PRICE', 'AWARDED_QTY', 'AWARDED_AMT', 'ADJUSTED_QTY', 'ADJUSTED_AMT', 'COMPLETED_QTY', 'COMPLETED_AMT']]
Pay_DF.to_csv('Contract_PayItems.csv')
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-08 15:04:43
2018-11-08 15:04:51


In [8]:
#HSIP Contracts
print(strftime("%Y-%m-%d %H:%M:%S"))
Cont_DF = pd.read_excel(DocumentsDir + '\\Cont_DF.xlsx')
print(Cont_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-11-09 09:09:29
(905, 33)
2018-11-09 09:09:30


In [9]:
C0 = set(Cont_DF[(Cont_DF.Status=='C') & 
        (Cont_DF.F_AwardDate>datetime(2007,1,1,0,0)) & 
        (Cont_DF.F_CompDate <datetime(2015,1,1,0,0)) &
        (Cont_DF.HSIPAmount>0)]['ContNum'].astype(str))
C1 = set(Cont_DF.ContNum.astype(str))
C2 = set(Pay_DF.ContNum.astype(str))
C3 = set(Desc_DF.ContNum.astype(str))
C4 = set(Let_DF.CONTNUM.astype(str))
C5 = set(Doc_DF.ContNum.astype(str))
df = pd.DataFrame(columns=['Selected','HSIP','Amounts','Descriptions','LetItems','Documents'],
                  index  =['Selected','HSIP','Amounts','Descriptions','LetItems','Documents'])
df.loc['Selected']     = [len(C0.intersection(s)) for s in [C0,C1,C2,C3,C4,C5]]
df.loc['HSIP']         = [len(C1.intersection(s)) for s in [C0,C1,C2,C3,C4,C5]]
df.loc['Amounts']      = [len(C2.intersection(s)) for s in [C0,C1,C2,C3,C4,C5]]
df.loc['Descriptions'] = [len(C3.intersection(s)) for s in [C0,C1,C2,C3,C4,C5]]
df.loc['LetItems']     = [len(C4.intersection(s)) for s in [C0,C1,C2,C3,C4,C5]]
df.loc['Documents']    = [len(C5.intersection(s)) for s in [C0,C1,C2,C3,C4,C5]]
df

,Selected,HSIP,Amounts,Descriptions,LetItems,Documents
Selected,387,387,387,63,280,383
HSIP,387,905,692,144,515,498
Amounts,387,692,692,143,515,497
Descriptions,63,144,143,2451,2085,1737
LetItems,280,515,515,2085,9006,6379
Documents,383,498,497,1737,6379,11383


In [ ]:
#create a list of documents to download and download them
#add available descriptions

In [10]:
Doc_DF1 = Doc_DF[(Doc_DF.ContNum.isin(C1)) & (Doc_DF.Type=='pdf')]
Doc_DF1 = Doc_DF1.sort_values(['ContNum','Type'])
Doc_DF1.index = Doc_DF1.ContNum
Doc_DF1.shape

(24428, 7)

In [ ]:
import socket 
socket.setdefaulttimeout(120)
for c in list(set(Doc_DF1.index)):
    print(c,Doc_DF1.loc[c].shape[0])
    dir_path = ReceivedDir + '\\Plans\\{}'.format(c)
    if not os.path.exists(dir_path):
        print('Make Directory: ' + dir_path)
        os.mkdir(dir_path)
    for l,f in zip(list(Doc_DF1[Doc_DF1.ContNum==c]['Link']),list(Doc_DF1[Doc_DF1.ContNum==c]['Filename'])):
        if not os.path.exists(dir_path + '\\' + f):
            try:
                hsmpy3.common.Downloadfile(URL=l,OutputDir=dir_path,extension='pdf')
            except:
                print('Failed: {}, {}'.format(c,l))
                pass
        else:
            print('Already Exist: '+ dir_path + '\\' + f)